In [2]:
import sys
import argparse
from spectral_cube import SpectralCube
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import glob

In [3]:
def plot_subcubes(coord_subcubes, ls='-', color=None, lw=1):
    for i, coord in  enumerate(coord_subcubes):
        xlo, ylo, xhi, yhi = coord
        plt.plot([xlo, xhi, xhi, xlo, xlo],
                 [ylo, ylo, yhi, yhi, ylo],
                 color=color, ls=ls, lw=lw)
        plt.annotate(f'{i}',
                     xy=(np.mean([xlo, xhi]),
                         np.mean([ylo, yhi]))
                     )
def plot_border(wcs, N):
    brc = wcs.pixel_to_world(0,0,0)[0]
    trc = wcs.pixel_to_world(0,N,0)[0]
    tlc = wcs.pixel_to_world(N,N,0)[0]
    blc = wcs.pixel_to_world(N,0,0)[0]
    plt.plot([brc.ra.deg, trc.ra.deg, tlc.ra.deg, blc.ra.deg, brc.ra.deg],
             [brc.dec.deg, trc.dec.deg, tlc.dec.deg, blc.dec.deg, brc.dec.deg],
            'k-', lw=4)

def plot_grid(coord_subcubes):
    ## Plot grid of subcubes
    #plt.subplot(projection=wcs, slices=['x','y', 0])  # It is not correcly projecting the pixels when using this mode. Alternatively, I could use APLpy
    plt.subplot()
    #plot_border(wcs, N)
    plot_subcubes(coord_subcubes)
    #plt.grid()
    plt.xlabel('R.A. [deg]')
    plt.ylabel('Dec. [deg]')
    plt.gca().invert_xaxis()
    #plt.savefig(grid_plot, bbox_inches='tight', dpi=200)



In [4]:
coord_subcubes = np.loadtxt('results/coord_subcubes.csv', skiprows=1, delimiter=',')
plot_subcubes(coord_subcubes)
plt.gca().invert_xaxis()

In [5]:
df = pd.read_csv('results/final_catalog.csv', delimiter=' ')
df_truth = pd.read_csv('/mnt/scratch/sdc2/data/development_large/sky_ldev_truthcat_v2.txt', delimiter=' ')

In [6]:
fig, axs = plt.subplots(2, 4, figsize=(20, 12))

for i, c in enumerate(['ra', 'dec', 'hi_size', 'line_flux_integral', 'central_freq', 'pa', 'i', 'w20']):
    ax = axs.flatten()[i]

    #sns.kdeplot(data=df_truth, x=c, color='r', lw=2, ax=ax)
    ax.set_title(c)
    if c in ['hi_size', 'line_flux_integral', 'w20']:
        ax.set_xscale('log')
        ax.set_yscale('log')
        sns.histplot(data=df, x=c, kde=False, log_scale=True, edgecolor='None', ax=ax)
        sns.histplot(data=df_truth, bins=20, x=c, kde=False, ax=ax, alpha=0.1, edgecolor='0.2', facecolor='0.8',  log_scale=True, element="step")
    else:
        sns.histplot(data=df, x=c, kde=False, edgecolor='None', ax=ax)
        sns.histplot(data=df_truth, bins=50, x=c, kde=False,  ax=ax, alpha=0.9, edgecolor='0.2', facecolor='0.8',  element="step")
        ax.set_yscale('log')

fig.savefig("results/plots/plot1.png", bbox_inches='tight')

In [7]:
fig, ax = plt.subplots(1, 1, figsize=(10, 8))

c = 'hi_size'
ax.set_xscale('log')
sns.histplot(data=df, x=c, edgecolor='None', kde=False, log_scale=True, ax=ax)
sns.histplot(data=df_truth, bins=20, x=c, kde=False, ax=ax, alpha=0.1, edgecolor='k', facecolor='0.8',  log_scale=True, element="step")
ax.set_yscale('log')

In [8]:
fig, ax = plt.subplots(1, 1, figsize=(10, 8))

c = 'w20'
ax.set_xscale('log')
sns.histplot(data=df, x=c, edgecolor='None', kde=False, log_scale=True, ax=ax)
sns.histplot(data=df_truth, bins=20, x=c, kde=False, ax=ax, alpha=0.1, edgecolor='k', facecolor='0.8',  log_scale=True, element="step")
ax.set_yscale('log')

In [9]:
print(f"{np.mean(df['w20'])}+/-{np.std(df['w20'])}")
print(f"{np.mean(df_truth['w20'])}+/-{np.std(df_truth['w20'])}")

In [12]:
fig, ax = plt.subplots(1, 1, figsize=(12, 12))

plot_subcubes(coord_subcubes)
#ax.plot(df_truth['ra'], df_truth['dec'], '+r')
ax.plot(df['ra'], df['dec'], '+k')
plt.gca().invert_xaxis()


In [11]:
cat_files = sorted(glob.glob('results/sofia/*/subcube_*_final_catalog.csv'))
for cat_file in cat_files:
    with open(cat_file, 'r') as f:
        lines = f.readlines()
        print(cat_file, len(lines))